In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

import os


In [27]:
def get_dat_dir_path()->str:
    return os.path.abspath('../dat')
    
def get_chk_dir_path()->str:
    return os.path.abspath('../chk')

def get_train_csv_path()->str:
    dat_dir = get_dat_dir_path()
    # print(dat_dir)
    return os.path.join(dat_dir, "train.csv")

def get_test_csv_path()->str:
    dat_dir = get_dat_dir_path()
    return os.path.join(dat_dir, "test_x.csv")

def get_sample_sub_path()->str:
    dat_dir = get_dat_dir_path()
    return os.path.join(dat_dir, "sample_submission.csv")


In [28]:
train = pd.read_csv(get_train_csv_path())
test = pd.read_csv(get_test_csv_path())
sample_submission = pd.read_csv(get_sample_sub_path())

In [29]:
# preprocesscing

In [30]:
def alpha_num(txt:str)->str:
    return re.sub(r"[^A-Za-z0-9 ]", "", txt)

train["text"] = train["text"].apply(alpha_num)

In [31]:
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [32]:
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

def remove_stopwords(text:str) -> str:
    final_txt = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_txt.append(i.strip())
    return " ".join(final_txt)

In [33]:
train["text"] = train["text"].apply(alpha_num).apply(remove_stopwords)
test["text"] = test["text"].apply(alpha_num).apply(remove_stopwords)

In [34]:
train

,index,text,author
0,0,almost choking much much wanted say strange ex...,3
1,1,sister asked suppose,2
2,2,engaged one day walked perusing Janes last let...,1
3,3,captain porch keeping carefully way treacherou...,4
4,4,mercy gentlemen odin flung hands Dont write an...,3
...,...,...,...
54874,54874,Mr Smith odin whispered hardly dared hope come,2
54875,54875,told plan captain us settled details accomplis...,4
54876,54876,sincere wellwisher friend sister LUCY odin,1
54877,54877,wanted lend money,3


In [35]:
# x_train = train["text"].values

In [36]:
# x_train

In [37]:
x_train = np.array([x for x in train["text"]])
x_test = np.array([x for x in test["text"]])
y_train = np.array([x for x in train["author"]])

In [38]:
# x_train

In [39]:
# Modeling

In [40]:
vocab_size = 20000
embedding_dim = 128
max_length = 500
padding_type = "post"

In [41]:
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index

In [42]:
list(word_index)[:10]

['odin', 'not', 'said', 'no', 'one', 'mr', 'will', 'upon', 'now', 'man']

In [43]:
train_sequence = tokenizer.texts_to_sequences(x_train)
train_padded = pad_sequences(train_sequence, padding=padding_type, maxlen=max_length)

test_sequence = tokenizer.texts_to_sequences(x_test)
test_padded = pad_sequences(test_sequence, padding=padding_type, maxlen=max_length)

In [44]:
train_padded

array([[  141,  7259,    20, ...,     0,     0,     0],
       [  217,    58,   221, ...,     0,     0,     0],
       [  682,     5,    59, ...,     0,     0,     0],
       ...,
       [ 2407, 19738,   126, ...,     0,     0,     0],
       [  316,  3532,   164, ...,     0,     0,     0],
       [  203,     2,   811, ...,     0,     0,     0]], dtype=int32)

In [45]:
############################################################################
## this model part is from 텐서플로2와 머신러닝으로 시작하는 자연어처리 04. 텍스트분류
############################################################################

model_name = "cnn_classifier_en"
BATCH_SIZE=512
NUM_EPOCHS=10
VALID_SPLIT=0.1
MAX_LEN=train_padded.shape[1]

kargs = {
    "model_name": model_name,
    "vocab_size": vocab_size,
    "embedding_size": 128,
    "num_filters": 100,
    "dropout_rate": .5,
    "hidden_dimension": 250,
    "output_dimension": 5
}

In [46]:
class CNNClassifier(tf.keras.Model):
    def __init__(self, **kargs):
        super(CNNClassifier, self).__init__(name=kargs["model_name"])
        self.embedding = tf.keras.layers.Embedding(input_dim=kargs["vocab_size"],
                                                   output_dim=kargs["embedding_size"])
        self.conv_list = [ tf.keras.layers.Conv1D(filters=kargs["num_filters"],
                                                  kernel_size=kernel_size,   
                                                  padding="valid", 
                                                  activation="relu", 
                                                  kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.))
                            for kernel_size in [3,4,5] ]
        self.pooling = tf.keras.layers.GlobalMaxPooling1D()
        self.dropout = tf.keras.layers.Dropout(kargs["dropout_rate"])
        self.fc1 = tf.keras.layers.Dense(units=kargs["hidden_dimension"], activation="relu", 
                                         kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.)
                                         )
        self.fc2 = tf.keras.layers.Dense(units=kargs["output_dimension"], activation="softmax",
                                         kernel_constraint=tf.keras.constraints.MaxNorm(max_value=3.)
                                         )


    def call(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        x = tf.concat([self.pooling(conv(x)) for conv in self.conv_list], axis=-1)
        x = self.fc1(x)
        x = self.fc2(x)

        return x                                         


In [47]:
model = CNNClassifier(**kargs)


In [48]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
model.build(train_padded.shape)
print(model.summary())            

Model: "cnn_classifier_en"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  2560000   
_________________________________________________________________
conv1d_3 (Conv1D)            multiple                  38500     
_________________________________________________________________
conv1d_4 (Conv1D)            multiple                  51300     
_________________________________________________________________
conv1d_5 (Conv1D)            multiple                  64100     
_________________________________________________________________
global_max_pooling1d_1 (Glob multiple                  0         
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple            

In [51]:
#num_epochs = 20
earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", 
                                                    min_delta=0.0001, 
                                                    patience=2)

checkpoint_path = os.path.join( get_chk_dir_path(), model_name, "weights.h5" )
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                monitor="val_accuracy", 
                                                verbose=1, 
                                                save_best_only=True, 
                                                save_weights_only=True)

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir, exist_ok=True)

history = model.fit(train_padded, y_train, 
                    batch_size=BATCH_SIZE, 
                    epochs=NUM_EPOCHS, 
                    verbose=2, 
                    validation_split=VALID_SPLIT, 
                    callbacks=[earlystop_callback, cp_callback])

Epoch 1/10

Epoch 00001: val_accuracy improved from -inf to 0.55977, saving model to /Users/daewonyoon/github/daewonyoon/misc/2020/10_dacon_predict_novelist/chk/cnn_classifier_en/weights.h5
97/97 - 147s - loss: 1.4240 - accuracy: 0.3940 - val_loss: 1.1249 - val_accuracy: 0.5598
Epoch 2/10

Epoch 00002: val_accuracy improved from 0.55977 to 0.71228, saving model to /Users/daewonyoon/github/daewonyoon/misc/2020/10_dacon_predict_novelist/chk/cnn_classifier_en/weights.h5
97/97 - 142s - loss: 0.8717 - accuracy: 0.6728 - val_loss: 0.7690 - val_accuracy: 0.7123
Epoch 3/10

Epoch 00003: val_accuracy improved from 0.71228 to 0.73998, saving model to /Users/daewonyoon/github/daewonyoon/misc/2020/10_dacon_predict_novelist/chk/cnn_classifier_en/weights.h5
97/97 - 142s - loss: 0.6001 - accuracy: 0.7825 - val_loss: 0.7035 - val_accuracy: 0.7400
Epoch 4/10

Epoch 00004: val_accuracy did not improve from 0.73998
97/97 - 146s - loss: 0.4740 - accuracy: 0.8296 - val_loss: 0.7075 - val_accuracy: 0.7394
E

In [52]:
pred = model.predict(test_padded)

In [53]:
pred

array([[1.9904643e-03, 1.2089707e-02, 9.2557865e-01, 5.2187432e-02,
        8.1537263e-03],
       [9.5090695e-02, 7.0567065e-01, 9.4645815e-03, 3.8130987e-02,
        1.5164304e-01],
       [4.1340652e-01, 5.8451283e-01, 1.0949204e-03, 1.7250596e-04,
        8.1317773e-04],
       ...,
       [1.0811018e-03, 9.9793917e-01, 7.3060196e-06, 9.6551189e-04,
        6.8933018e-06],
       [1.6137594e-02, 9.8251778e-01, 5.3815992e-04, 5.2907073e-04,
        2.7739056e-04],
       [9.5007634e-01, 5.6357193e-03, 4.9050138e-03, 6.6119875e-03,
        3.2770913e-02]], dtype=float32)

In [54]:
pred.shape

(19617, 5)

In [55]:
test_padded.shape

(19617, 500)

In [56]:
sample_submission[[str(i) for i in range(5)]] = pred
sample_submission

,index,0,1,2,3,4
0,0,0.001990,0.012090,0.925579,0.052187,0.008154
1,1,0.095091,0.705671,0.009465,0.038131,0.151643
2,2,0.413407,0.584513,0.001095,0.000173,0.000813
3,3,0.058792,0.000792,0.730282,0.015804,0.194329
4,4,0.608498,0.012198,0.004230,0.334077,0.040998
...,...,...,...,...,...,...
19612,19612,0.000437,0.999486,0.000003,0.000058,0.000016
19613,19613,0.019042,0.001783,0.000779,0.000022,0.978375
19614,19614,0.001081,0.997939,0.000007,0.000966,0.000007
19615,19615,0.016138,0.982518,0.000538,0.000529,0.000277


In [57]:
sample_submission.to_csv("submission.csv", index=False, encoding="utf-8")